# BK figures
This code will create the figures for BK model reduction using MBAM for finding boundaries, and manual model reduction

## BK full model reduction

In [ ]:
# The code used to produce the output from reducing the 8 parameter MBAM model to 3 parameters
#= Commented out
include("BK_functions/bk_setup_script.jl"); 
outp = MBAM(BK_simulator, phi0, x_grid, model_iters=[10, 11, 12, 13, 14],
       move_dir=[-1, -1, 1, 1, -1], boundary_time=10, verbose=1, reduce_func=BK_reduce);
=#

# Saved into file  Figures/BK_manual_model_10_to_14_reduct.jld", "outp"

In [8]:
using PlotlyJS
plt = PlotlyJS.plot(scatter(;y=randn(3)))


In [7]:
# Load the saved data
using JLD
outp = load("BK_manual_model_10_to_14_reduct.jld", "outp");

# The output is an array containing [phis, costs, Ds, Vs, t_out, y_out, model_reduced]
# each of those contain all information for each iteration, so out[1][3] would be phi values of the 3rd model iteration


(
5x2 Array{Any,2}:
 [-13.027053197600004,-0.8675005677047232,-2.276917346245468,-0.544727175441672,-10.151948911834628,1.8180767775454285,3.414442608412176,0.6931471805599453]  …  [-19.73885126878325,-10.158929504340358,-2.9637617908962697,-0.552377240968135,-10.292018512456947,2.194872804239878,5.94676248434345,0.10578562617249801]
 [-19.73885126878325,-2.9637617908962697,-0.552377240968135,-10.292018512456947,2.194872804239878,5.94676248434345,0.10578562617249801]                          [-35.248626537528054,-2.9364251907799703,-0.6035821743583822,-10.348966074661599,5.3961898012232705,9.888208259727053,-3.223448891936689]                 
 [-2.9364251907799703,-0.6035821743583822,-10.348966074661599,-3.415966833158743,1.0760516253450394,5.588707742445324]                                           [-2.9445748207349203,-0.6196420735233441,-10.340822436745757,-7.388601480469609,1.1050337204312926,9.565673665930234]                                     
 [-2.9445748207349203,-0.6196420

In [1]:
# Load the BK functions and the x_grid(Ca, V)
include("../BK_functions/bk_setup_script.jl"); 
Ca = reshape(hcat(x_grid...)[1,:], (8,13));
V = reshape(hcat(x_grid...)[2,:], (8,13));

### Plot model behaviour
Original 8 parameter vs 4 parameter model

In [ ]:
# Plot the model output at phi0
using PlotlyJS
using NBInclude
nbinclude("MBAM_plots.ipynb")
import MBAM_plots

f_model(phi; model_id=0) = reshape(BK_simulator(phi0, hcat(x_grid...),model_id=model_id), (8,13));

p_orig = MBAM_plots.plot(V', f_model(phi0)', 
    marker_color=Any[1:13, "rgb(16,32,77)"],
    mode=["markers", "markers+lines","lines"]
    )

p_iter5 = MBAM_plots.plot(V', reshape(BK_simulator(exp(outp[1][5,1]), hcat(x_grid...),model_id=14), (8,13))');

a = PlotlyJS.Plot(p_orig, MBAM_plots.layouts())
b = PlotlyJS.Plot(p_iter5, MBAM_plots.layouts())



PlotlyJS.plot([a, b])



In [ ]:
typeof(["rgb(0, $g, 0)" for g in 100:20:(100+7*20)])

In [ ]:
using PlotlyJS

In [ ]:
p1 = Plot(scatter(;y=randn(3)))
p2 = Plot(histogram(;x=randn(50), nbinsx=4))
p3 = Plot(scatter(;y=cumsum(randn(12)), name="Random Walk"))
p4 = Plot([scatter(;x=1:4, y=[0, 2, 3, 5], fill="tozeroy"),
           scatter(;x=1:4, y=[3, 5, 1, 7], fill="tonexty")])

typeof(scatter(;y=randn(3)))

### Plot parameter behaviour during boundary search

In [5]:
nbinclude("MBAM_plots.ipynb")

function create_boundary_iter_plot(outp; param_names::AbstractArray=Any[], model_id=collect(10:14))
    # From the output of the MBAM code, creates pretty plots automatically
    
    if isempty(param_names)
        param_names = Array(Array{AbstractString},size(outp[1],1));
        param_names[1] = ["L0", "zL", "J0", "zJ", "Kd", "C", "D", "E"]
        param_names[2] = ["L0", "J0", "zJ", "Kd", "C", "D", "E"]
        param_names[3] = ["J0", "zJ", "Kd", "L0^1/4 * C"," L0^1/4 * D", "E / L0^1/4"];
        param_names[4] = ["J0", "zJ", "Kd", "C * E"," L0^1/4 * D"];
    end
    
    
    color_list_Ca = ["rgb(0, $g, 0)" for g in 100:20:(100+7*20)]
    color_list_phi = ["rgb($r, 0, 0)" for r in 100:20:(100+7*20)]
    
    
    name_list_Ca = ["[Ca]=$(round(Ca[i1]*1e6)) &mu;mol" for i1 in 1:size(Ca,1)]
    
    model_outp_plots = Array(PlotlyJS.Plot{PlotlyJS.GenericTrace{Dict{Symbol,Any}}}, size(outp[1],1));
    boundary_iter_plots = Array(PlotlyJS.Plot{PlotlyJS.GenericTrace{Dict{Symbol,Any}}}, size(outp[1],1));
    
    
    for i1 = 1:size(outp[1],1)
        model_outp_plots[i1] = 
            PlotlyJS.Plot(
                MBAM_plots.plot(V', reshape(BK_simulator(exp(outp[1][i1,1]), hcat(x_grid...),
                    model_id=model_id[i1]), size(V))',
        marker_color=color_list_Ca, showlegend=(i1==1), legendgroup="model", name = name_list_Ca
                ),
                MBAM_plots.layouts()
            )
        
        t_out = outp[5][i1];
        y_out = hcat(outp[6][i1]...)
        
        @show param_names[i1]
        @show name_list_Ca
        
        #=
        boundary_iter_plots[i1] = PlotlyJS.Plot(
            MBAM_plots.plot(t_out, y_out[1:div(size(y_out,1),2),:]',
        showlegend=true, legendgroup="phi_iter_$(i1)", name=["L0", "zL", "J0", "zJ", "Kd", "C", "D", "E"],
        mode="markers", marker_color=color_list_phi, marker_size=3
        ),
            MBAM_plots.layouts("BK_MBAM_search")
        )
        =#
    end
                
    
    return model_outp_plots, boundary_iter_plots
end


        
    

create_boundary_iter_plot (generic function with 1 method)

In [6]:
model_outp_plots, boundary_iter_plots = create_boundary_iter_plot(outp)

LoadError: LoadError: UndefVarError: outp not defined
while loading In[6], in expression starting on line 1

In [ ]:
# Now actually plot it
plt = PlotlyJS.plot([model_outp_plots[1] boundary_iter_plots[1]
    model_outp_plots[2] boundary_iter_plots[2]
    model_outp_plots[3] boundary_iter_plots[3]
    model_outp_plots[4] boundary_iter_plots[4]
    model_outp_plots[5] boundary_iter_plots[5]])

relayout!(plt, height=1600)

plt

In [ ]:
# Plotting parameter value versus time - iter 1
param_names = Array(Any,length(outp[1]));
param_names[1] = ["L0", "zL", "J0", "zJ", "Kd", "C", "D", "E"]'
bla = MBAM_plots.plot(outp[5][1],hcat(outp[6][1]...)[1:8,:]') #t vs all

PlotlyJS.plot(bla, MBAM_plots.layouts("BK_MBAM_search"))

In [ ]:
# Plotting parameter value versus time - iter 2
param_names[2] = ["L0", "J0", "zJ", "Kd", "C", "D", "E"]'
Plots.plot(outp[5][2],hcat(outp[6][2]...)[1:7,:]', xlabel="time", label=param_names[2]) #t vs all

In [ ]:
# Plotting parameter value versus time - iter 3
param_names[3]=["J0", "zJ", "Kd", "L0^1/4 * C"," L0^1/4 * D", "E / L0^1/4"]';
Plots.plot(outp[5][3],hcat(outp[6][3]...)[1:6,:]', xlabel="time", label=param_names[3]) #t vs all

In [ ]:
# Plotting parameter value versus time - iter 4
param_names[4]=["J0", "zJ", "Kd", "C * E"," L0^1/4 * D"]';
Plots.plot(outp[5][4],hcat(outp[6][4]...)[1:5,:]', xlabel="time", xlims=(0,10), label=param_names[4]) #t vs all

In [ ]:
# Now plot the geodesic courses, from iteration n1, for parameter p1 being fix and plotting the rest
#Iteration one
Plots.plot(hcat(outp[6][1]...)[[1; 3:8],:]', hcat(outp[6][1]...)[2,:]', 
xlabel="log(\phi_i)", ylabel="log(zL)", xlim=(-20,6), label=param_names[1][[1;3:8]']) #all vs zL

In [ ]:
#Iteration two
param_names[2] = ["L0", "J0", "zJ", "Kd", "C", "D", "E"]'
Plots.plot(hcat(outp[6][2]...)[[2:7],:]', hcat(outp[6][2]...)[1,:]', 
xlabel="log(\phi_i)", ylabel="log(L0)", label=param_names[2][[2:7]']) #all vs L0

In [ ]:
#Iteration three
param_names[3]=["J0", "zJ", "Kd", "L0^1/4 * C"," L0^1/4 * D", "E / L0^1/4"]';
Plots.plot(hcat(outp[6][3]...)[[1:5],:]', hcat(outp[6][3]...)[6,:]', 
xlabel="log(\phi_i)", ylabel="log(E / L0^1/4)", xlims=(-11,10), label=param_names[3][[1:5]']) #all vs L0

In [ ]:
#Iteration four
param_names[4]=["J0", "zJ", "Kd", "C * E"," L0^1/4 * D"]';
Plots.plot(hcat(outp[6][4]...)[[2:5],:]', hcat(outp[6][4]...)[1,:]', 
xlabel="log(\phi_i)", ylabel="log(J0)", xlims=(-11,10), label=param_names[4][[2:5]']) #all vs L0